**Installing the Dash Library**

In [1]:
#Installing the Dash library using PIP
!pip install dash

**Importing Libraries**

In [2]:
#Importing libraries
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

**Reading in Data**

In [3]:
#Reading in the data
Data = pd.read_csv("Theme Park Queue Times.csv")

**Code for Figure 2 (Average Queue Times Grouped by Operator)**

Run this code prior to running the code for the rest of the dashboard, as this declared figure is referenced within the main dashboard code.

In [4]:
#Grouping the data by theme park operator and specifying key descriptive statistics
QueuesGroupedByOperator = Data.groupby(by="Operator")["Queue Time"].describe()
#Making a bar chart of the mean queue time per operator
Figure2 = px.bar(QueuesGroupedByOperator, x=QueuesGroupedByOperator.index, y="mean", color_discrete_sequence=["#8b0000"])
#Updating the layout of the figure to add axis titles, a plot title and new colours, amongst other settings
Figure2.update_layout(xaxis={"categoryorder" : "total descending",
                             "tickangle" : 30,
                             "title" : "Operator"},
                      yaxis={"title" : "Average Queue Time (minutes)"},
                      title="Major Attraction Queue Times by Operator",
                      paper_bgcolor="#87cefa",
                      plot_bgcolor="#87cefa",
                      font={"family" : "Roboto",
                            "color" : "#00008b"})

**Code for the Dashboard and Location-Based Figures**

Run this after running the code block above, but before attempting to run the dashboard in the code block below. This code declares all of the necessary code for the dashboard itself, including callbacks, the location-based figures and their associated interactivity.

In [5]:
#Declaring a list of location options for the interactive location plot
LocationOptions = ["Country", "Continent"]

#Declaring the dashboard itself
Dashboard = dash.Dash(__name__)

#Declaring the dashboard itself, with a main Div encompassing an interval item and a sub-Div that fades in, encompassing everything else
#The sub-Div encompasses the text and figures and is styled in unison with the Plotly figures
Dashboard.layout = html.Div(style={"backgroundColor" : "#87cefa",
                                   "fontFamily" : "Roboto",
                                   "color" : "#00008b"}, children=[dcc.Interval(
    id="PageInterval",
    interval=100,
    n_intervals=0,
    max_intervals=5,
    disabled=False),
    html.Div(id="container",
 children=[
    html.H1("Various factors affect the average queue times for top attractions in theme parks"),
    html.H2("One key factor is location"),
    html.P("At both a country and continent level, there are differences in top attraction queue times at theme parks by location. Brazil tops the tables in terms of country queue times, with Japan and Hong Kong not far behind, and parks in South America and Asia generally have longer queue times on average. On the other end of the spectrum, Belgium, Austria and Denmark's top attraction queue times are the lowest, and parks in Europe have the shortest queue times on average."),
    html.P("The data can be broken down by either country or continent; choose an option below:"),
    dcc.Dropdown(id="LocationGranularity",
                 options=[{
                     "label": i,
                     "value": i
                 } for i in LocationOptions]),
    dcc.Graph(id="figure1"),
    html.H2("Another key factor is operator"),
    html.P("Universal and Disney, the two park operators whose parks are most attended on average, top the tables in terms of operator queue times. On the other end of the spectrum, Plopsa and Looping Group have the lowest queue times for top attractions on average."),
    dcc.Graph(id="bar-graph2", figure=Figure2),
    html.Div(["All queue time data is sourced from the Parks index on ", html.A("queue-times.com", href="https://queue-times.com/parks", target="_blank"), "."])
], style={"opacity" : 0, "transition" : "opacity 2s ease-in-out"})])

#Declaring a callback to allow the user to choose whether they want the data shown by country or continent
#Depending on the user's choice, a figure aggregating the data by country or continent will be shown
@Dashboard.callback(
    Output("figure1", "figure"),
    Input("LocationGranularity", "value")
)
def UpdateGraph(Location):
  #If the user chooses Continent, a bar chart aggregating the data by continent is shown
  if Location == "Continent":
    #Grouping the descriptive statistics by continent
    QueuesGroupedByContinent = Data.groupby(by="Continent")["Queue Time"].describe()
    #Drawing the figure using the mean queue time by continent
    figure = px.bar(QueuesGroupedByContinent, x=QueuesGroupedByContinent.index, y="mean", color_discrete_sequence=["#00008b"])
    #Making some layout adjustments, including changing the category order and tick angle, adding titles and changing the colour
    figure.update_layout(xaxis={"categoryorder" : "total descending",
                                "tickangle" : 30},
                          yaxis={"title" : "Average Queue Time (minutes)"},
                          title="Major Attraction Queue Times by Continent",
                          paper_bgcolor="#87cefa",
                          plot_bgcolor="#87cefa",
                          font={"family" : "Roboto",
                                "color" : "#00008b"})
  #If the user chooses Country or does not play with the interactive dropdown menu, a bar chart aggregating the data by country is shown
  else:
    #Grouping the descriptive statistics by country
    QueuesGroupedByCountry = Data.groupby(by="Country")["Queue Time"].describe()
    #Drawing the figure using the mean queue time by country
    figure = px.bar(QueuesGroupedByCountry, x=QueuesGroupedByCountry.index, y="mean", color_discrete_sequence=["#008000"])
    #Making some layout adjustments, including changing the category order and tick angle, adding titles and changing the colour
    figure.update_layout(xaxis={"categoryorder" : "total descending",
                                "tickangle" : 30},
                          yaxis={"title" : "Average Queue Time (minutes)"},
                          title="Major Attraction Queue Times by Country",
                          paper_bgcolor="#87cefa",
                          plot_bgcolor="#87cefa",
                          font={"family" : "Roboto",
                                "color" : "#00008b"})
  #Returning the chosen figure
  return figure

#Declaring a callback to make the content fade in when the page is opened, using the interval item as an input
@Dashboard.callback(
    Output("container", "style"),
    Input("PageInterval", "n_intervals")
)
def PageFadeIn(n_intervals):
  #Returning the style necessary to make the material fade in
  return {"opacity" : min(n_intervals * 0.2, 1), "transition" : "opacity 2s ease-in-out"}

**Running the Dashboard**

Execute this code block to run the dashboard.

In [6]:
#Running the dashboard
if __name__ == '__main__':
    Dashboard.run(debug=True)


<IPython.core.display.Javascript object>